In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pykrige

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 852.6/852.6 kB 7.6 MB/s eta 0:00:00


In [4]:
# Import libraries to use
from datetime import datetime
import pykrige
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
import h5py
import xarray as xr
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from pykrige import variogram_models, UniversalKriging, OrdinaryKriging

In [8]:
def getKNearest(fromX, fromY, measuringPtX, measuringPtY, k):
  # returns a list of k indices of pts from fromX and fromY (assumed to correspond to x and y coordinates of the same pts) that are the nearest to measuringPtX and measuringPtY
  distances = np.sqrt((fromX - measuringPtX)**2 + (fromY - measuringPtY)**2)
  # Get the indices of the k-nearest points
  k_nearest_indices = np.argsort(distances)[:k]
  return k_nearest_indices

# Redirect the standard output to the log file
#sys.stdout = open(log_file_name, "w")


"""# Define functions"""

def visualizePointsInBox(allTrackX, allTrackY, xCoordMin, xCoordMax, yCoordMin, yCoordMax):
  fig, ax = plt.subplots()
  #ax.scatter(allSurfX, allSurfY, label='Surface points', s=0.2)
  ax.scatter(allTrackX, allTrackY, label='Track bed points', s=0.2)
  # Plot the batches
  # Vertical lines
  ax.axvline(x=xCoordMin, color="black")
  ax.axvline(x=xCoordMax, color="black")
  ax.axhline(yCoordMin, color="black")
  ax.axhline(yCoordMax, color="black")

  ax.set_xlabel('X')
  ax.set_ylabel('Y')
  ax.set_title('Surface data vs track data: batches visualized')
  #ax.legend(loc = "lower right")

  plt.show()
# Store the direction (degrees) of each vector (surf_vx, surf_vy)
def getVectorDir(x, y):
    # Calculate the angle in radians
    #angle_rad = np.arctan2(np.abs(y), x)
    angle_rad = np.arctan2(y,x)
    # Convert the angle from radians to degrees
    angle_deg = np.degrees(angle_rad)

    # Ensure the angle is within the range [0, 360)
    angle_deg = (angle_deg + 360) % 360

    #return abs(angle_deg -90) # Subtract 90 b/c 0 degrees = up in the coordinate system used by pykrige. also it just wants the axis, so we constrain to abs
    return angle_deg -90

In [24]:
# -*- coding: utf-8 -*-
"""krigingResidualsPyKrige.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1nif5cTGIZkoKRQRzI-CV61f2Hv7xPNJG

# Settings
"""

# Box settings
SQUAREBOXWIDTH = 50 # m
EXPANDSTEP = 10 # m
SHRINKSTEP = EXPANDSTEP
MAX_NUM_PTS = 80
MIN_NUM_PTS = 15

# Which variogram models to even try in order to find the best fit
METHODS = ['exponential', 'power', 'linear', 'gaussian', 'spherical']

# Pick average of 3 nearest pts' angles from velocity vectors
ANISOTROPY_ANGLE_METH = "KNN"
K = 1

# Which subset of dataset to run on
START_INDEX = 0
END_SLICE_INDEX = 40

TRACKBED_XIND = 7
TRACKBED_YIND = 8
TRACKBED_TARGETIND = 9

# Display plots of variograms?
PLOT = False
RUN_COLAB = False

VERBOSE_OUTPUT = False

import sys
'''
if (len(sys.argv) > 1):
  START_INDEX = int(sys.argv[1])
  END_SLICE_INDEX = int(sys.argv[2])
  print("recieved args: ", START_INDEX, " ", END_SLICE_INDEX)
'''
TRACKBED_XIND = 7
TRACKBED_YIND = 8
TRACKBED_TARGETIND = 9

# Display plots of variograms?
PLOT = False
RUN_COLAB = True

if RUN_COLAB:
  pass
else:
  log_file_name = "output_log_" + "(" + str(START_INDEX) + "," + str(END_SLICE_INDEX) + ")" + ".txt"
  sys.stdout = open(log_file_name, "w")


#angle = getVectorDir(-1,-1)
#print(angle)

"""# Import data"""

if RUN_COLAB:
  df_merged = pd.read_csv('/content/drive/MyDrive/Predicted and preprocessed data sets /training_test_combined_vmag_ky_230710.csv')
else:
  df_merged =  pd.read_csv('training_test_combined_vmag_ky_230710.csv')
df_merged.columns

"""# Data Preprocessing"""

# Remove duplicates by setting to mean
df2 = df_merged.groupby(['track_bed_x', 'track_bed_y']).agg({'surf_x':'mean', 'surf_y':'mean', 'surf_vx':'mean', 'surf_vy': 'mean', 'surf_elv': 'mean', 'surf_dhdt':'mean', 'surf_SMB': 'mean', 'track_bed_x':'first', 'track_bed_y':'first', 'track_bed_target':'mean', 'v_mag':'mean'})
print("num duplicates found: ", len(df_merged) - len(df2))
df_merged = df2.reset_index(drop=True)
df_merged.columns
print("Number of rows after duplicates removed: ", df_merged.shape)

"""# Execute kriging"""

trackDatTmp = np.array(df_merged)
trackDatTmp.shape
trackDat_x = trackDatTmp[:, TRACKBED_XIND]
trackDat_y = trackDatTmp[:, TRACKBED_YIND]
trackDat_angle = getVectorDir(trackDatTmp[:, 2], trackDatTmp[:, 3])
len(trackDat_angle)

trackDat = np.column_stack((trackDatTmp, trackDat_angle))
trackDat.shape

# For each point in tracking point dataset
  # Grab the track bed data points that are within  within box (except for that that point)
  # Print size of points grabbed in order to ensure it is reasonable
  # train kriging based on that dataset

boxWidth = SQUAREBOXWIDTH
outputCols = ['surf_x', 'surf_y', 'surf_vx', 'surf_vy','surf_elv', 'surf_dhdt', 'surf_SMB', 'track_bed_x', 'track_bed_y', 'v_mag', 'track_bed_target', 'residual', 'prediction']
out = np.empty( (0, 13) ) # (track_bed_x, track_bed_y, target_prediction, target_actual, residual)

slice = df_merged[START_INDEX:END_SLICE_INDEX].copy()
print("\t".join(str(value) for value in outputCols))
Kriging_startime = datetime.now()
for index, row in slice.iterrows():
  #print("Pixel " + str(index + 1) + " out of " + str(df_merged.shape[0]) )
  # Hold the prediction and ground truth for track_bed_target at the current pixel
  predictionThisRow = None
  groundTruthThisRow = row['track_bed_target']

  trackRowsToTrainOn = None

  #print("Trying to find a good box size...")
  xMin, xMax, yMin, yMax = (None, None, None, None)
  expandStep = EXPANDSTEP
  shrinkStep = SHRINKSTEP
  previousProblem = 0 # 1 = too big; -1 = too small
  while True:
    #print("Top of loop")
    xMin, xMax = (row['track_bed_x'] - boxWidth, row['track_bed_x'] + boxWidth)
    yMin, yMax = (row['track_bed_y'] - boxWidth, row['track_bed_y'] + boxWidth)

    # We are still fiting kriging on the entire dataset (not just the slice)
    x_selected_ind = np.where((trackDat_x >= xMin) & (trackDat_x <= xMax) )[0]
    y_selected_ind = np.where((trackDat_y >= yMin) & (trackDat_y <= yMax) )[0]
    #visualizePointsInBox(trackDat_x, trackDat_y, xMin, xMax, yMin, yMax)

    # Select indices that are in selected x and y range (inside the rectangle) to init object on
    indicesXY = list(set(x_selected_ind).intersection(y_selected_ind))
    trackRowsToTrainOn = np.array([trackDat[ind] for ind in indicesXY])

    # If none were selected, perform same code as if some but too few were selected
    if (len(trackRowsToTrainOn) < MIN_NUM_PTS):

      # Prevent overshooting by reducing step size
      if (previousProblem == 1): # Was too big last time
        expandStep /= 2
      else:
        pass#expandStep *= 1.5

      boxWidth += expandStep
      previousProblem = -1
      continue

    # Remove the row corresponding to the center point
    mask = np.logical_and(trackRowsToTrainOn[:, TRACKBED_XIND] == row['track_bed_x'], trackRowsToTrainOn[:, TRACKBED_YIND] == row['track_bed_y'])
    trackRowsToTrainOn = trackRowsToTrainOn[~mask] # Grab track rows that exclude it
    #print(trackRowsToTrainOn)

    # Print number of points
    numPts = len(trackRowsToTrainOn)
    #print(numPts)

    # Perform iterative resizing of box to ensure correct length
    if (numPts < MIN_NUM_PTS):

      # Prevent overshooting by reducing step size
      if (previousProblem == 1): # Was too big last time
        expandStep /= 2
      else:
        pass#expandStep *= 1.5

      boxWidth += expandStep
      previousProblem = -1
    elif (numPts > MAX_NUM_PTS):

      # Prevent overshooting by reducing step size
      if (previousProblem == -1): # Was too small last time
        shrinkStep /= 2
      else:
        pass#shrinkStep *= 1.5

      boxWidth -= shrinkStep
      previousProblem = 1
    else:
      break # We have chosen an appropriate number of trackRowsToTrainOn

  #print("Num pts to fit on: ", numPts)
  #print("Appropriate box selected; points extracted to trackRowsToTrainOn.")

  # Fit the variogram model
  #print("Fit variogram model...")
  nlags = len(trackRowsToTrainOn[:, 1])
  # Choose an appropriate anisotropy angle using the direction of ice flow velocity
  #print('angles:')
  #print(trackRowsToTrainOn[:, 11])
  # For angle determination
  kNearest_PtsToTrainOn_indices = getKNearest(trackRowsToTrainOn[:, TRACKBED_XIND], trackRowsToTrainOn[:, TRACKBED_YIND], row['track_bed_x'], row['track_bed_y'], K)
  angles = trackDat_angle[kNearest_PtsToTrainOn_indices]
  #print("angles: ", angles)
  anisotropy_angle = angles.mean()
  #anisotropy_angle = 90


  linear = UniversalKriging(trackRowsToTrainOn[:, TRACKBED_XIND], trackRowsToTrainOn[:, TRACKBED_YIND], trackRowsToTrainOn[:, TRACKBED_TARGETIND], variogram_model="linear", anisotropy_scaling = 3, anisotropy_angle = anisotropy_angle, enable_plotting=PLOT, nlags=nlags)
  exponential = UniversalKriging(trackRowsToTrainOn[:, TRACKBED_XIND], trackRowsToTrainOn[:, TRACKBED_YIND], trackRowsToTrainOn[:, TRACKBED_TARGETIND], variogram_model="exponential",anisotropy_scaling = 3, anisotropy_angle = anisotropy_angle, enable_plotting=PLOT, nlags=nlags)
  power = UniversalKriging(trackRowsToTrainOn[:, TRACKBED_XIND], trackRowsToTrainOn[:, TRACKBED_YIND], trackRowsToTrainOn[:, TRACKBED_TARGETIND], variogram_model="power", anisotropy_scaling = 3, anisotropy_angle = anisotropy_angle, enable_plotting=PLOT, nlags=nlags)
  gaussian = UniversalKriging(trackRowsToTrainOn[:, TRACKBED_XIND], trackRowsToTrainOn[:, TRACKBED_YIND], trackRowsToTrainOn[:, TRACKBED_TARGETIND], variogram_model="gaussian",anisotropy_scaling = 3, anisotropy_angle = anisotropy_angle, enable_plotting=PLOT, nlags=nlags)
  spherical = UniversalKriging(trackRowsToTrainOn[:, TRACKBED_XIND], trackRowsToTrainOn[:, TRACKBED_YIND], trackRowsToTrainOn[:, TRACKBED_TARGETIND], variogram_model="spherical",anisotropy_scaling = 3, anisotropy_angle = anisotropy_angle, enable_plotting=PLOT, nlags=nlags)

  # Get the Q2 scores for each model
  model_names = ['linear', 'exponential', 'power', 'gaussian', 'spherical']
  models = [linear, exponential, power,  gaussian, spherical]

  #print(['surf_x', 'surf_y', 'surf_vx', 'surf_vy','surf_elv', 'surf_dhdt', 'surf_SMB', 'track_bed_x', 'track_bed_y', 'v_mag', 'residual'])
  stats = []
  for i, model in enumerate(models):
    #print(model_names[i])
    #print(model.get_epsilon_residuals())
    #model.print_statistics()
    stats.append(model.get_statistics()[2])
    #stats.append(np.mean(np.abs(model.get_epsilon_residuals())))
    #print(np.mean(np.abs(model.get_epsilon_residuals())))

  #stats = [linear.get_statistics()[1], exponential.get_statistics()[1], power.get_statistics()[1], gaussian.get_statistics()[1], spherical.get_statistics()[1]]
  #print(stats)

  # Choose the best model...if there is a tie, look at other accuracy metrics, then if tied on all three, just use the first of the tied ones.

  # Get the index in stats of the one with the Q2 score that is the closest to 1
  ind_closest_to_one = np.abs(np.array(stats)).argmin()
  kriging = models[ind_closest_to_one]
  #print("Best variogram: ", model_names[ind_closest_to_one])



  #print("Making prediction for current pixel...")
  prediction, predictionVariance = kriging.execute("points", row['track_bed_x'], row['track_bed_y'])
  predictionThisRow = prediction[0]

  #print("Calculating residual of prediction...")
  residualThisRow = groundTruthThisRow - predictionThisRow

  #print("Saving results for this data point to dataframe...")
  existingRowData = row[['surf_x', 'surf_y', 'surf_vx', 'surf_vy','surf_elv', 'surf_dhdt', 'surf_SMB', 'track_bed_x', 'track_bed_y', 'v_mag', 'track_bed_target']].copy() # Is this a copy...?
  existingRowData['residual'] = residualThisRow
  existingRowData['prediction'] = predictionThisRow
  #print(existingRowData)
  print("\t".join(str(value) for value in existingRowData.values.tolist()))
  out = np.vstack( (out, np.array(existingRowData)))

Kriging_stoptime = datetime.now()
print("---------------------------------------------------------")
print(f"Finished applying kriging to pixels: {START_INDEX}:{END_SLICE_INDEX}")
print("Time to run: ", Kriging_stoptime - Kriging_startime)

df_krigout = pd.DataFrame(columns = outputCols, data=out)

if RUN_COLAB:
  pass
  #df_krigout.to_csv(f"/content/drive/MyDrive/Kriging/Kriging-ProfWang'sNewMethod/outputs/with angle-pixels (" + str(START_INDEX) + "," + str(END_SLICE_INDEX) + ") residuals.csv")
else:
  df_krigout.to_csv( "./pixels (" + str(START_INDEX) + "," + str(END_SLICE_INDEX) + ") residuals.csv" )
  sys.stdout.close()
  sys.stdout = sys.__stdout__
  print(f"Finished applying kriging to pixels: {START_INDEX}:{END_SLICE_INDEX}")
  print("Time to run: ", Kriging_stoptime - Kriging_startime)



num duplicates found:  94
Number of rows after duplicates removed:  (632612, 11)
surf_x	surf_y	surf_vx	surf_vy	surf_elv	surf_dhdt	surf_SMB	track_bed_x	track_bed_y	v_mag	track_bed_target	residual	prediction
-278075.0	-1860125.0	-28.47723	10.470569	952.47925	-1.0990746	-0.6505264	-278074.96875	-1860189.125	30.341150994262907	355.8812866210937	10.949722515438395	344.9315641056553
-278075.0	-1843625.0	-217.74167	6.631936	971.1958	-1.2553866	-0.64528835	-278074.875	-1843669.25	217.8426437350984	414.4259643554688	-2.565611962385276	416.9915763178541
-278075.0	-1840025.0	-229.13731	114.66887	964.4169	-1.7170284	-0.7125957	-278074.8125	-1839996.875	256.22813385948274	61.30860900878906	-1.836115949020602	63.144724957809665
-278075.0	-1841075.0	-237.63937	75.172966	975.3123	-1.5823828	-0.67396367	-278074.65625	-1841013.5	249.24575220298956	84.89076232910156	-1.9968461614515576	86.88760849055312
-278075.0	-1850525.0	-220.97133	-63.44889	941.261	-0.9963478	-0.8764233	-278074.53125	-1850554.5	229.9

In [25]:
def rmspe(y_true, y_pred):
    return np.sqrt(np.nanmean(np.square(((y_true - y_pred) / y_true))))*100
def rmspe_1(y_true, y_pred):
    return np.sqrt(np.nanmean(np.square(y_true - y_pred) / y_true))*100
#metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
# from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score


In [26]:
# Calculate statistics
df_kout = df_krigout
print('RMSE:',np.sqrt(mean_squared_error(df_kout['track_bed_target'], df_kout['prediction'])))
print('RMSE Percentage:',rmspe(df_kout['track_bed_target'], df_kout['prediction']))
print('Mean Absolute Error:', mean_absolute_error(df_kout['track_bed_target'], df_kout['prediction']))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_kout['track_bed_target'], df_kout['prediction']))
print('R^2 Score:', r2_score(df_kout['track_bed_target'], df_kout['prediction']))

RMSE: 14.054230327579885
RMSE Percentage: 6.0715109646909875
Mean Absolute Error: 8.875521842931269
Mean Absolute Percentage Error: 0.035442615009204356
R^2 Score: 0.997213521736916
